In [1]:
!pip install langchain
!pip install langgraph
!pip install -qU langchain[google-genai]
!pip install allosaurus
!pip install gTTS
!pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 1.6 MB/s  0:00:09 eta 0:00:010:00:01


In [2]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

Enter API key for Google Gemini:  ········


In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
chat_prompt_template = ChatPromptTemplate.from_messages(
    [("system", "Act as an expert Chinese language tutor with over 10 years of experience. Use the following communication style: encouraging, patient, culturally sensitive and systematically progressive. Gently correct mistakes (including pronounciation mistakes) in real time. Regularly highlight student achievements and improvements to maintain motivation. You are tutoring a native US English speaker. When speaking Chinese, never use vocabulary about the pre-2021 HSK-3 level under any circumstances.",), MessagesPlaceholder(variable_name="messages"),]
)

In [4]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = chat_prompt_template.invoke(state["messages"])
    response = model.invoke(prompt)
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

#Adding Memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [5]:
from langchain_core.messages import HumanMessage
from gtts import gTTS
from io import BytesIO
from pygame import mixer

config = {"configurable": {"thread_id": "CM"}}
mp3_file_like = BytesIO()
pygame.mixer.init()
while True:
  user_input = input("You>:")
  input_messages = [HumanMessage(user_input)]
  output = app.invoke({"messages": input_messages}, config)
  last_message = output["messages"][-1]
  print("Teacher>:", end="")
  last_message.pretty_print()
  # Use gTTS and pygame to say the AI message with Taiwanese voice
  tts = gTTS(text=last_message.text(), lang='zh-TW', slow=True)
  tts.write_to_fp(mp3_file_like)
  pygame.mixer.music.load(mp3_file_like)
  pygame.mixer.music.play()

/home/michael/.local/pipx/shared/lib/python3.11/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


NameError: name 'pygame' is not defined